# CS3405 Homework 2 - Dynamic Programming

## Student Name: 張子健
## Student ID:  103011230

### 1. Longest Common Subsequence

* Straight forward brute-force Approach

In [56]:
# x, y can be list or string
def lcs(x,y):
    if len(x) == 0 or len(y) == 0:
        return 0
    elif x[0] == y[0]:
        ans = 1 + lcs(x[1:], y[1:])
    else:
        ans = max(lcs(x, y[1:]), lcs(x[1:], y))
    return ans

In [57]:
lcs("TOYOTA", "TOKYO")

4

* Dictinary-based Approach

In [58]:
# x, y can be list or string, d is a dictionary
def lcsDict(x, y, d):
    key = str(len(x)) + "-" + str(len(y))
    # Check if it's in the Dict
    if key in d:
        return d[key]
    #Three different situations
    if len(x) == 0 or len(y) == 0:
        d[key] = 0
        return d[key]
    elif x[0] == y[0]:
        ans = 1 + lcsDict(x[1:], y[1:], d)
    else:
        ans = max(lcsDict(x, y[1:], d), lcsDict(x[1:], y, d))
    #Saved in Dict
    d[key] = ans
    return d[key]

In [59]:
lcsDict("TOYOTA", "TOKYO", {})

4

* Array-based Approach

In [60]:
import numpy as np
def lcsArray(x, y):
    a = np.zeros([len(x)+1, len(y)+1])
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            if x[i-1] == y[j-1]:
                a[i, j] = 1 + a[i-1, j-1]
            else:
                a[i, j] = max(a[i, j-1], a[i-1, j])
    return int(a[len(x), len(y)])

In [61]:
lcsArray("TOYOTA", "TOKYO")

4

### 2. Chain Matrix Multiplication

* Straight forward brute-force Approach

In [62]:
def cmm(chain):
    if len(chain)<=1:
        return 0
    else:
        #which index to cut
        cost = min(cmm(chain[k:]) + cmm(chain[:k]) + chain[0][0] * chain[k][0] * chain[-1][1] for k in range(1, len(chain)))
    return cost

In [63]:
ch = [(1, 2), (2, 3), (3, 1), (1, 5)]
cmm(ch)

13

* Dictinary-based Approach

In [64]:
def cmmDict(chain, d, i, j):
    key = str(i) + "-" + str(j)   #start index to end index
    if key in d:
        return d[key]
    if len(chain)<=1:
        d[key] = 0
        return d[key]
    else:
        cost = min(cmmDict(chain[k:], d, i+k, j) + cmmDict(chain[:k], d, i, i+k-1) + chain[0][0] * chain[k][0] * chain[-1][1] for k in range(1, len(chain)))
    d[key] = cost
    return d[key]

In [65]:
ch = [(1, 2), (2, 3), (3, 1), (1, 5)]
d={}
cmmDict(ch, d, 0, len(ch)-1)

13

* Array-based Approach

In [66]:
def cmmArray(chain):
    a = np.zeros([len(chain), len(chain)])
    for i in range(1, len(chain)):
        x, y = 0, i
        for j in range(1, len(chain)-i+1):
            a[x][y] = min(a[x][k] + a[k+1][y] + chain[x][0] * chain[k+1][0] * chain[y][1] for k in range(x, y))
            x, y = x+1, y+1    #Move right and down
    return a

In [67]:
ch = [(1, 2), (2, 3), (3, 1), (1, 5)]
cmmArray(ch)

array([[ 0.,  6.,  8., 13.],
       [ 0.,  0.,  6., 16.],
       [ 0.,  0.,  0., 15.],
       [ 0.,  0.,  0.,  0.]])

### 3. Text Justification

* Straight forward brute-force Approach

In [68]:
def tj(tl, lw):
    if len(tl) == 0:
        return 0
    elif len(tl) == 1:
        return (lw - len(tl[0])) ** 2
    else:
        usage = len(tl[0])
        waste = [(lw-usage)**2]
        maxwords = 1
        while usage + 1 + len(tl[maxwords]) <= lw:
            usage = usage + 1 + len(tl[maxwords])
            waste.append((lw-usage)**2)
            maxwords += 1
            if maxwords == len(tl):
                break
    return min([waste[k] + tj(tl[k+1:], lw) for k in range(maxwords)])

In [34]:
t = "Dolen says these findings could help scientists better understand social behavior, as well as give clues about possible treatments for serotonin-related human conditions like schizophrenia and PTSD. "
textList = t.split()
tj(textList, 40)

399

* Dictinary-based Approach

In [69]:
def tjDict(tl, lw, d, i, j):
    key = str(i)+"-"+str(j)
    if key in d:
        return d[key]
    elif i >= j:
        d[key] = 0
        return d[key]
    else:
        usage = len(tl[i])
        waste = [(lw-usage)**2]
        k = 1
        while (i + k < j and usage + 1 + len(tl[i+k]) <= lw):
            usage = usage + 1 + len(tl[i+k])
            waste.append((lw - usage)**2)
            k += 1
        d[key] = min([w + tjDict(tl, lw, d, i+r+1, j) for r, w in enumerate(waste)])
        return d[key]

In [70]:
d = {}
tjDict(textList, 40, d, 0, len(textList))

399

* Array-based Approach

In [79]:
def tjArray(textlist, lw):
    textlen = len(textlist)
    cost = [0] * (textlen + 1)
    wordsinline = cost[:]
    
    for i in range(textlen-1, -1, -1):
        usage = len(textlist[i])
        waste = [(lw - len(textlist[i])) ** 2]
        j = i + 1
        
        while (j <= textlen - 1 and usage + 1 + len(textlist[j]) <= lw):
            usage = usage + 1 + len(textlist[j])
            waste.append((lw - usage) ** 2)
            j += 1
         
        cost[i],wordsinline[i] = min([(w + cost[i+1+k], k+1) for k, w in enumerate(waste)])
    print ("Cost :", cost[0])
    
    jtext = ""
    i = 0
    while i <= textlen-1:
        line = str(textlist[i])
        if i < textlen-1:
            for j in range(i+1, i+wordsinline[i]):
                line += " " + str(textlist[j])
        line += "\n"
        jtext += line
        i += wordsinline[i]
    print (jtext)

In [80]:
tjArray(textList, 40)

Cost : 399
Dolen says these findings could
help scientists better understand
social behavior, as well as give
clues about possible treatments for
serotonin-related human conditions
like schizophrenia and PTSD.



### 4. Knapsack Problem
#### In this Knapsack problem we have a knapsack that can hold a specific weight (C) and we have a series of objects to place in it. Each object has a weight (wt) and a corresponding value (V). Our goal is best utilize the space in the knapsack by maximizing the value of the objects placed in it.

* Straight forward brute-force Approach

In [73]:
def ks(C, wt, V, n):
    
    if n == 0 or C == 0 :
        return 0
    elif (wt[n-1] > C):
        return ks(C , wt , V , n-1)
    else:
        return max(V[n-1] + ks(C-wt[n-1] , wt , V , n-1), ks(C , wt , V , n-1))

In [74]:
ks(50, [10, 20, 25, 7, 12, 15] , [60, 100, 120, 40, 70, 90] , 6)

270

* Dictinary-based Approach

In [75]:
def ksDict(C, wt, V, n, d):
    key = str(n) + "-" + str(C)
    if key in d:
        return d[key]
    if n == 0 or C == 0 :
        d[key] = 0
        return d[key]
    elif (wt[n-1] > C):
        return ksDict(C , wt , V , n-1, d)
    else:
        value = max(V[n-1] + ksDict(C-wt[n-1] , wt , V , n-1, d), ksDict(C , wt , V , n-1, d))
    d[key] = value
    return d[key]

In [76]:
ksDict(50, [10, 20, 25, 7, 12, 15] , [60, 100, 120, 40, 70, 90] , 6, {})

270

* Array-based Approach

In [77]:
def ksArray(C, wt, V, n):
    a = [[0 for x in range(C+1)] for x in range(n+1)]
    for i in range(n+1):
        for j in range(C+1):
            if i==0 or j==0:
                a[i][j] = 0
            elif wt[i-1] <= j:
                a[i][j] = max(V[i-1] + a[i-1][j-wt[i-1]],  a[i-1][j])
            else:
                a[i][j] = a[i-1][j] 
    return a[n][C]

In [78]:
ksArray(50, [10, 20, 25, 7, 12, 15] , [60, 100, 120, 40, 70, 90] , 6)

270